In [ ]:
!apt-get install -y pciutils lshw
!lspci | grep -i nvidia

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids usb.ids
The following NEW packages will be installed:
  libpci3 lshw pci.ids pciutils usb.ids
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 883 kB of archives.
After this operation, 3,256 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 lshw amd64 02.19.git.2021.06.19.996aaad9c7-2build1 [321 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/main amd64 usb.ids all 2022.04.02-1 [219 kB]
Fetched 883 kB in 1s (659 kB/s)
Selecting previously unselected package pci.ids.
(Reading d

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
import subprocess
import time

# Start the Ollama server
subprocess.Popen(["ollama", "serve"])

# Wait for a few seconds for Ollama to load
time.sleep(3)

In [ ]:
# Set the MODEL variable
ollama_model_name = "gemma2:9b"
# ollama_model_name = "gemma:2b"


# Run the ollama pull command
!ollama pull gemma2:9b         # model demo
#!ollama pull "gemma:2b"              # model dùng đánh giá


pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling ff1d1fc78170...   0% ▕▏    0 B/5.4 GB                  pulling manifest 
pulling ff1d1fc78170...   0% ▕▏    0 B/5.4 GB                  pulling manifest 
pulling ff1d1fc78170...   0% ▕▏    0 B/5.4 GB                  pulling manifest 
pulling ff1d1fc78170...   0% ▕▏ 2.1 MB/5.4 GB                  pulling manifest 
pulling ff1d1fc78170...   0% ▕▏  18 MB/5.4 GB                  pulling manifest 
pulling ff1d1fc78170...   1% ▕▏  50 MB/5.4 GB                  pulling manifest 
pulling ff1d1fc78170...   2% ▕▏ 100 MB/5.4 GB                  pulling manifest 
pulling ff1d1fc78170...   2% ▕▏ 126 MB/5.4 GB                  pulling manifest 
pulling ff1d1fc78170...   3% ▕▏ 145 MB/5.4 GB                  pulling manifest 
pullin

In [ ]:
%pip install instructor ipywidgets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 51.6 MB/s eta 0:00:00


In [ ]:
import os
import json
import instructor
from pydantic import BaseModel, Field
from enum import Enum
from typing import List
from openai import OpenAI

class PhishingProbability(str, Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"

class SuspiciousElement(BaseModel):
    element: str
    reason: str

class SimplePhishingAnalysis(BaseModel):
    is_potential_phishing: bool   #true/false
    phishing_probability: PhishingProbability
    suspicious_elements: List[SuspiciousElement]
    recommended_actions: List[str]
    explanation: str

def analyze_email(email_content: str) -> SimplePhishingAnalysis:
    client = instructor.from_openai(
        OpenAI(
            base_url="http://127.0.0.1:11434/v1", # local sever port 11434
            api_key="ollama", # mặc định
        ),
        mode=instructor.Mode.JSON,
    ) # định nghĩa LLM

    resp = client.chat.completions.create(
            model=ollama_model_name,
            messages=[
                {
                    "role": "system",
                    "content": "Analyze the provided email content and metadata to determine if it's a potential phishing attempt. Provide your analysis in a structured format matching the SimplePhishingAnalysis model.",
                },
                {
                    "role": "user",
                    "content": email_content,
                },
            ],
            response_model=SimplePhishingAnalysis,
        )
    return resp

In [ ]:
from IPython.display import display, HTML
import ipywidgets as widgets

# GUI components
email_input = widgets.Textarea(
    value='',
    placeholder='Paste the email content here...',
    description='Email:',
    disabled=False,
    layout={'width': '100%', 'height': '200px'}
)

analyze_button = widgets.Button(
    description='Analyze Email',
    disabled=False,
    button_style='primary',
    tooltip='Click to analyze the email',
    icon='check'
)

output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        print("Analyzing email...")
        try:
            analysis = analyze_email(email_input.value)
            display(HTML(format_analysis(analysis)))
        except Exception as e:
            print(f"An error occurred: {str(e)}")

analyze_button.on_click(on_button_clicked)

def format_analysis(analysis: SimplePhishingAnalysis) -> str:
    color = "red" if analysis.is_potential_phishing else "green"
    result = f"""
    <h2 style="color: {color};">{'Potential Phishing Detected' if analysis.is_potential_phishing else 'Likely Legitimate Email'}</h2>
    <p><strong>Phishing Probability:</strong> {analysis.phishing_probability.value}</p>
    <h3>Suspicious Elements:</h3>
    <ul>
    """
    for element in analysis.suspicious_elements:
        result += f"<li><strong>{element.element}:</strong> {element.reason}</li>"
    result += "</ul>"
    result += f"""
    <h3>Recommended Actions:</h3>
    <ul>
    """
    for action in analysis.recommended_actions:
        result += f"<li>{action}</li>"
    result += "</ul>"
    result += f"<h3>Explanation:</h3><p>{analysis.explanation}</p>"
    return result

# Display the GUI
display(email_input, analyze_button, output)

Textarea(value='', description='Email:', layout=Layout(height='200px', width='100%'), placeholder='Paste the e…

Button(button_style='primary', description='Analyze Email', icon='check', style=ButtonStyle(), tooltip='Click …

Output()

In [ ]:
""" Legit email
From: jennifer.smith@company.com
To: team@company.com
Subject: Quarterly Review Meeting - Thursday, 2 PM

Dear Team,

I hope this email finds you well. I'm writing to remind everyone about our upcoming Quarterly Review Meeting scheduled for this Thursday at 2 PM in the main conference room.

Agenda:
1. Q2 Performance Overview
2. Project Updates
3. Q3 Goals and Strategies
4. Open Discussion

Please come prepared with your team's updates and any questions you may have. If you're unable to attend in person, you can join via our usual video conferencing link:
https://company.zoom.us/j/123456789

Don't forget to bring your laptops for the interactive portion of the meeting.

If you have any questions or need to discuss anything before the meeting, feel free to drop by my office or send me an email.

Looking forward to seeing everyone on Thursday!

Best regards,
Jennifer Smith
Head of Operations
Company Inc.
"""

""" Phishing email
From: security@amazonsupport.net
To: john.doe@company.com
Subject: Urgent: Your Amazon Account Has Been Locked

Dear Valued Customer,

We have detected unusual activity on your Amazon account. To prevent unauthorized access, we have temporarily locked your account for your protection.

To unlock your account and restore full access, please click on the link below and verify your information:

https://amaz0n-account-verify.com/unlock-account

If you do not verify your account within 24 hours, it will be permanently disabled.

Thank you for your prompt attention to this matter.

Best regards,
Amazon Security Team

This email was sent by Amazon.com. To ensure delivery to your inbox, please add security@amazonsupport.net to your address book.
"""




' Phishing email\nFrom: security@amazonsupport.net\nTo: john.doe@company.com\nSubject: Urgent: Your Amazon Account Has Been Locked\n\nDear Valued Customer,\n\nWe have detected unusual activity on your Amazon account. To prevent unauthorized access, we have temporarily locked your account for your protection.\n\nTo unlock your account and restore full access, please click on the link below and verify your information:\n\nhttps://amaz0n-account-verify.com/unlock-account\n\nIf you do not verify your account within 24 hours, it will be permanently disabled.\n\nThank you for your prompt attention to this matter.\n\nBest regards,\nAmazon Security Team\n\nThis email was sent by Amazon.com. To ensure delivery to your inbox, please add security@amazonsupport.net to your address book.\n'